In [1]:
import pandas
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

In [2]:
data = "labourR-data/occupations_en.csv"
isco_df = pandas.read_csv("labourR-data/ISCOGroups_en.csv")
df = pandas.read_csv(data)

In [3]:
df.shape

(2942, 14)

In [4]:
df["raw_text"] = df["preferredLabel"] + " " +  df["altLabels"]
df["raw_text"].fillna(value=df["preferredLabel"], inplace=True)
df["raw_text"] = df["raw_text"].str.replace("\n", " ")
#df = df[["function", "raw_text"]]

In [23]:
df[df.raw_text.isnull()]

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,raw_text,similarity_score


In [24]:
df

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,raw_text,similarity_score
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2166,technical director,technical and operations director\nhead of tec...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2166.4.7,technical director technical and operations di...,0.188934
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine technician\nmetal drawin...,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4,metal drawing machine operator metal drawing m...,0.138119
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3,precision device inspector inspector of precis...,0.077074
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1,air traffic safety technician air traffic safe...,0.123475
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9,hospitality revenue manager hospitality revenu...,0.313887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,demographer,demography research analyst\ndemography studie...,NaN,released,2016-07-05T16:21:57Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Demographers study a variety of parameters rel...,2120.2,demographer demography research analyst demogr...,0.023468
2938,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,sorter labourer,grader\nyard labourer\nrecycler\nrecycling sit...,NaN,released,2017-01-17T13:22:45Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Sorter labourers sort recyclable materials and...,9612.2,sorter labourer grader yard labourer recycler ...,0.051575
2939,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,armoured car guard,armoured truck escort\ntruck escort\narmoured ...,NaN,released,2016-12-22T10:57:58Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Armoured car guards ensure the safe transporta...,5414.1.2,armoured car guard armoured truck escort truck...,0.179745
2940,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,civil service administrative officer,government administrative officer\nlocal autho...,NaN,released,2016-07-05T16:17:26Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Civil service administrative officers perform ...,2422.1,civil service administrative officer governmen...,0.253923


In [6]:
df["raw_text"].isnull().value_counts()


False    2942
Name: raw_text, dtype: int64

In [7]:
model = SentenceTransformer('all-MiniLM-L12-v1')

In [8]:
sentence_embeddings = model.encode(df.raw_text.tolist())

In [9]:
sentence_embeddings.shape

(2942, 384)

In [10]:
input_sentence = "Data scientist"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
460,data scientist,data scientist data research scientist researc...,0.759876,2511,Professionals,Information and communications technology prof...,Software and applications developers and analysts,Systems analysts
2491,data analyst,data analyst data analysts data warehousing an...,0.726391,2511,Professionals,Information and communications technology prof...,Software and applications developers and analysts,Systems analysts
2065,data entry supervisor,data entry supervisor data processing coordina...,0.605775,3341,Technicians and associate professionals,Business and administration associate professi...,Administrative and specialised secretaries,Office supervisors
670,information manager,information manager information officer academ...,0.602442,2622,Professionals,"Legal, social and cultural professionals","Librarians, archivists and curators",Librarians and related information professionals
2092,database developer,database developer database programmer databas...,0.592954,2521,Professionals,Information and communications technology prof...,Database and network professionals,Database designers and administrators
2640,chief data officer,chief data officer chief analytics officer CDO...,0.586012,1330,Managers,Production and specialised services managers,Information and communications technology serv...,Information and communications technology serv...
594,call centre analyst,call centre analyst trainee data analyst call ...,0.582058,3341,Technicians and associate professionals,Business and administration associate professi...,Administrative and specialised secretaries,Office supervisors
1196,data entry clerk,data entry clerk data-entry clerk data steward...,0.576400,4132,Clerical support workers,General and keyboard clerks,Keyboard operators,Data entry clerks
2646,computer scientist,computer scientist ICT researcher IT scientist...,0.572901,2511,Professionals,Information and communications technology prof...,Software and applications developers and analysts,Systems analysts
892,ICT business analysis manager,ICT business analysis manager IT business anal...,0.572349,2511,Professionals,Information and communications technology prof...,Software and applications developers and analysts,Systems analysts


In [11]:
input_sentence = "Private investor"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
1218,venture capitalist,venture capitalist seed capital investors fina...,0.627545,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
909,investor relations manager,investor relations manager investor relations ...,0.509619,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
686,investment adviser,investment adviser financial and investment ad...,0.493839,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
2909,investment clerk,investment clerk investment administrator inve...,0.461764,4312,Clerical support workers,Numerical and material recording clerks,Numerical clerks,"Statistical, finance and insurance clerks"
2189,investment fund manager,investment fund manager investments manager po...,0.460390,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
2389,mutual fund broker,mutual fund broker mutual fund sales agent mut...,0.441407,3311,Technicians and associate professionals,Business and administration associate professi...,Financial and mathematical associate professio...,Securities and finance dealers and brokers
1516,real estate investor,real estate investor real estate entrepreneurs...,0.439797,3334,Technicians and associate professionals,Business and administration associate professi...,Business services agents,Real estate agents and property managers
185,investment manager,investment manager investment consultant globa...,0.433496,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
414,financial planner,financial planner wealth adviser financial adv...,0.425628,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
1923,private chef,private chef private cook private culinary exp...,0.410120,3434,Technicians and associate professionals,"Legal, social, cultural and related associate ...","Artistic, cultural and culinary associate prof...",Chefs


In [12]:
input_sentence = "Senior management consultant"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
1272,business consultant,business consultant business process consultant,0.672606,2421,Professionals,Business and administration professionals,Administration professionals,Management and organisation analysts
1162,healthcare consultant,healthcare consultant healthcare management ad...,0.635309,2422,Professionals,Business and administration professionals,Administration professionals,Policy administration professionals
726,business service manager,business service manager actuarial advisory se...,0.609763,1219,Managers,Administrative and commercial managers,Business services and administration managers,Business services and administration managers ...
2259,project manager,project manager project leader projects manage...,0.599071,1219,Managers,Administrative and commercial managers,Business services and administration managers,Business services and administration managers ...
2850,dating service consultant,dating service consultant matchmaking consulta...,0.598609,5169,Service and sales workers,Personal service workers,Other personal services workers,Personal services workers not elsewhere classi...
1151,business analyst,business analyst standardisation manager manag...,0.598378,2421,Professionals,Business and administration professionals,Administration professionals,Management and organisation analysts
563,sales account manager,sales account manager sales coordinator sales ...,0.597129,1420,Managers,"Hospitality, retail and other services managers",Retail and wholesale trade managers,Retail and wholesale trade managers
948,strategic planning manager,strategic planning manager strategic manager b...,0.588894,1213,Managers,Administrative and commercial managers,Business services and administration managers,Policy and planning managers
495,travel consultant,travel consultant business travel consultant t...,0.588610,4221,Clerical support workers,Customer services clerks,Client information workers,Travel consultants and clerks
1363,client relations manager,client relations manager client relationship m...,0.578595,2431,Professionals,Business and administration professionals,"Sales, marketing and public relations professi...",Advertising and marketing professionals


In [13]:
input_sentence = "Managing partner and director"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
1656,stage director,stage director theatre director scene director...,0.476910,2166,Professionals,Science and engineering professionals,"Architects, planners, surveyors and designers",Graphic and multimedia designers
1285,director of photography,director of photography lighting cameraman cin...,0.471830,2166,Professionals,Science and engineering professionals,"Architects, planners, surveyors and designers",Graphic and multimedia designers
2621,assistant stage director,assistant stage director first assistant direc...,0.467807,3435,Technicians and associate professionals,"Legal, social, cultural and related associate ...","Artistic, cultural and culinary associate prof...",Other artistic and cultural associate professi...
41,rooms division manager,rooms division manager director of rooms front...,0.463966,1411,Managers,"Hospitality, retail and other services managers",Hotel and restaurant managers,Hotel managers
1351,assistant video and motion picture director,assistant video and motion picture director di...,0.457638,3435,Technicians and associate professionals,"Legal, social, cultural and related associate ...","Artistic, cultural and culinary associate prof...",Other artistic and cultural associate professi...
2455,brokerage firm director,brokerage firm director brokerage firm chief e...,0.455791,1346,Managers,Production and specialised services managers,Professional services managers,Financial and insurance services branch managers
20,head chef,head chef industrial head chef chef de cuisine...,0.447549,3434,Technicians and associate professionals,"Legal, social, cultural and related associate ...","Artistic, cultural and culinary associate prof...",Chefs
2408,art director,art director advertising art director creative...,0.440462,2166,Professionals,Science and engineering professionals,"Architects, planners, surveyors and designers",Graphic and multimedia designers
2478,legal service manager,legal service manager legal office head head o...,0.434815,1349,Managers,Production and specialised services managers,Professional services managers,Professional services managers not elsewhere c...
563,sales account manager,sales account manager sales coordinator sales ...,0.434546,1420,Managers,"Hospitality, retail and other services managers",Retail and wholesale trade managers,Retail and wholesale trade managers


In [14]:
input_sentence = "Angel investor"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
1218,venture capitalist,venture capitalist seed capital investors fina...,0.702121,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
686,investment adviser,investment adviser financial and investment ad...,0.506546,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
2909,investment clerk,investment clerk investment administrator inve...,0.491283,4312,Clerical support workers,Numerical and material recording clerks,Numerical clerks,"Statistical, finance and insurance clerks"
909,investor relations manager,investor relations manager investor relations ...,0.480397,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
2189,investment fund manager,investment fund manager investments manager po...,0.475923,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
185,investment manager,investment manager investment consultant globa...,0.458797,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
1516,real estate investor,real estate investor real estate entrepreneurs...,0.457192,3334,Technicians and associate professionals,Business and administration associate professi...,Business services agents,Real estate agents and property managers
1745,investment fund management assistant,investment fund management assistant investmen...,0.456479,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers
2638,asset manager,asset manager wealth manager assets manager we...,0.449912,3311,Technicians and associate professionals,Business and administration associate professi...,Financial and mathematical associate professio...,Securities and finance dealers and brokers
442,corporate investment banker,corporate investment banker investment banking...,0.439123,2412,Professionals,Business and administration professionals,Finance professionals,Financial and investment advisers


In [36]:
input_sentence = "AR & VR engineer"

similarities = [util.cos_sim(model.encode(input_sentence), embedding)[0][0].item() for embedding in sentence_embeddings]
df["similarity_score"] = similarities

results = df[["preferredLabel", "raw_text", "similarity_score", "iscoGroup"]].sort_values(by="similarity_score", ascending=False).head(10)

results["CISCO_1"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
results["CISCO_2"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
results["CISCO_3"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
results["CISCO_4"] = results.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])
results

,preferredLabel,raw_text,similarity_score,iscoGroup,CISCO_1,CISCO_2,CISCO_3,CISCO_4
1484,video artist,video artist visuals artist interactive video ...,0.520073,2651,Professionals,"Legal, social and cultural professionals",Creative and performing artists,Visual artists
1556,aircraft maintenance engineer,aircraft maintenance engineer air transportati...,0.492553,3153,Technicians and associate professionals,Science and engineering associate professionals,Ship and aircraft controllers and technicians,Aircraft pilots and related associate professi...
106,integration engineer,integration engineer software integration engi...,0.477026,2511,Professionals,Information and communications technology prof...,Software and applications developers and analysts,Systems analysts
777,explosives engineer,explosives engineer ordnance engineer munition...,0.471576,2146,Professionals,Science and engineering professionals,Engineering professionals (excluding electrote...,"Mining engineers, metallurgists and related pr..."
2517,digital games designer,digital games designer virtual games designer ...,0.470771,2166,Professionals,Science and engineering professionals,"Architects, planners, surveyors and designers",Graphic and multimedia designers
2391,video designer,video designer projection & video designer gra...,0.469294,2166,Professionals,Science and engineering professionals,"Architects, planners, surveyors and designers",Graphic and multimedia designers
1421,commissioning engineer,commissioning engineer,0.465024,2149,Professionals,Science and engineering professionals,Engineering professionals (excluding electrote...,Engineering professionals not elsewhere classi...
80,video operator,video operator visual and graphic operator pro...,0.464288,3521,Technicians and associate professionals,Information and communications technicians,Telecommunications and broadcasting technicians,Broadcasting and audiovisual technicians
2878,metrologist,metrologist metrology technologist calibration...,0.460284,2112,Professionals,Science and engineering professionals,Physical and earth science professionals,Meteorologists
1414,audio-visual technician,audio-visual technician audiovisual assistant ...,0.458271,3521,Technicians and associate professionals,Information and communications technicians,Telecommunications and broadcasting technicians,Broadcasting and audiovisual technicians


In [35]:
isco_df[(isco_df.code >= 2400) & (isco_df.code <= 2500)]

,conceptType,conceptUri,code,preferredLabel,altLabels,inScheme,description
131,ISCOGroup,http://data.europa.eu/esco/isco/C2411,2411,Accountants,NaN,http://data.europa.eu/esco/concept-scheme/isco...,"Accountants plan, organize and administer acco..."
132,ISCOGroup,http://data.europa.eu/esco/isco/C2412,2412,Financial and investment advisers,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Financial and investment advisers develop fina...
133,ISCOGroup,http://data.europa.eu/esco/isco/C2413,2413,Financial analysts,NaN,http://data.europa.eu/esco/concept-scheme/isco...,Financial analysts conduct quantitative analys...
135,ISCOGroup,http://data.europa.eu/esco/isco/C2421,2421,Management and organisation analysts,NaN,http://data.europa.eu/esco/concept-scheme/isco...,Management and organization analysts assist or...
136,ISCOGroup,http://data.europa.eu/esco/isco/C2422,2422,Policy administration professionals,NaN,http://data.europa.eu/esco/concept-scheme/isco...,Policy administration professionals develop an...
137,ISCOGroup,http://data.europa.eu/esco/isco/C2423,2423,Personnel and careers professionals,NaN,http://data.europa.eu/esco/concept-scheme/isco...,Personnel and careers professionals provide pr...
138,ISCOGroup,http://data.europa.eu/esco/isco/C2424,2424,Training and staff development professionals,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Training and staff development professionals p...
140,ISCOGroup,http://data.europa.eu/esco/isco/C2431,2431,Advertising and marketing professionals,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Advertising and marketing professionals develo...
141,ISCOGroup,http://data.europa.eu/esco/isco/C2432,2432,Public relations professionals,NaN,http://data.europa.eu/esco/concept-scheme/occu...,"Public relations professionals plan, develop, ..."
142,ISCOGroup,http://data.europa.eu/esco/isco/C2433,2433,Technical and medical sales professionals (exc...,NaN,http://data.europa.eu/esco/concept-scheme/isco...,Technical and medical sales professionals (exc...


In [37]:
sentence_embeddings

array([[ 0.08251337,  0.05248352,  0.06106708, ..., -0.06386691,
         0.00786542,  0.03031025],
       [-0.10567711,  0.0577779 ,  0.00763851, ...,  0.00206289,
         0.00846538, -0.01200084],
       [-0.0009704 ,  0.13812947,  0.01281798, ...,  0.01551527,
         0.04177497,  0.00139211],
       ...,
       [-0.0260385 ,  0.09241939, -0.06050533, ...,  0.05273907,
         0.02820429, -0.01294944],
       [-0.00230223,  0.03144989,  0.03763039, ..., -0.08666099,
         0.03291652,  0.02399143],
       [-0.01887223,  0.04502025,  0.02102842, ...,  0.07298599,
         0.05076944,  0.01102118]], dtype=float32)

In [40]:
pandas.DataFrame(sentence_embeddings).to_csv("embeddings.tsv", sep="\t", index=False, header=None)

In [49]:
df["CISCO_1"] = df.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0])].preferredLabel.values[-1])
df["CISCO_2"] = df.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:2])].preferredLabel.values[-1])
df["CISCO_3"] = df.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:3])].preferredLabel.values[-1])
df["CISCO_4"] = df.iscoGroup.apply(lambda code: isco_df[isco_df.code == int(str(code)[0:4])].preferredLabel.values[-1])

df[["preferredLabel", "CISCO_1", "CISCO_2", "CISCO_3", "CISCO_4"]].to_csv("embeddings_metadata.tsv", sep="\t", index=False)

In [45]:
df

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,code,raw_text,similarity_score
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2166,technical director,technical and operations director\nhead of tec...,NaN,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,2166.4.7,technical director technical and operations di...,0.279607
1,Occupation,http://data.europa.eu/esco/occupation/000e93a3...,8121,metal drawing machine operator,metal drawing machine technician\nmetal drawin...,NaN,released,2016-07-05T17:09:43Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Metal drawing machine operators set up and ope...,8121.4,metal drawing machine operator metal drawing m...,0.237681
2,Occupation,http://data.europa.eu/esco/occupation/0019b951...,7543,precision device inspector,inspector of precision instruments\nprecision ...,NaN,released,2016-07-06T09:21:20Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Precision device inspectors make sure precisio...,7543.10.3,precision device inspector inspector of precis...,0.323486
3,Occupation,http://data.europa.eu/esco/occupation/0022f466...,3155,air traffic safety technician,air traffic safety electronics hardware specia...,NaN,released,2017-01-17T11:40:37Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Air traffic safety technicians provide technic...,3155.1,air traffic safety technician air traffic safe...,0.389466
4,Occupation,http://data.europa.eu/esco/occupation/002da35b...,2431,hospitality revenue manager,hospitality revenues manager\nyield manager\nh...,NaN,released,2017-01-17T13:33:42Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Hospitality revenue managers maximise revenue ...,2431.9,hospitality revenue manager hospitality revenu...,0.186548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2937,Occupation,http://data.europa.eu/esco/occupation/ff656b3a...,2120,demographer,demography research analyst\ndemography studie...,NaN,released,2016-07-05T16:21:57Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Demographers study a variety of parameters rel...,2120.2,demographer demography research analyst demogr...,0.101707
2938,Occupation,http://data.europa.eu/esco/occupation/ff8d4065...,9612,sorter labourer,grader\nyard labourer\nrecycler\nrecycling sit...,NaN,released,2017-01-17T13:22:45Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Sorter labourers sort recyclable materials and...,9612.2,sorter labourer grader yard labourer recycler ...,0.147452
2939,Occupation,http://data.europa.eu/esco/occupation/ffa4dd5d...,5414,armoured car guard,armoured truck escort\ntruck escort\narmoured ...,NaN,released,2016-12-22T10:57:58Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Armoured car guards ensure the safe transporta...,5414.1.2,armoured car guard armoured truck escort truck...,0.158029
2940,Occupation,http://data.europa.eu/esco/occupation/ffade2f4...,2422,civil service administrative officer,government administrative officer\nlocal autho...,NaN,released,2016-07-05T16:17:26Z,http://data.europa.eu/esco/regulated-professio...,NaN,NaN,http://data.europa.eu/esco/concept-scheme/occu...,Civil service administrative officers perform ...,2422.1,civil service administrative officer governmen...,0.194770
